In [20]:
import numpy as np
import cv2 as cv
from scipy.spatial.transform import Rotation

In [ ]:
# rvec_g2b: rotation of the gripper w.r.t. base (Rodrigues vec, radians)
rvec_g2b = [
    [0.0,      0.0,      0.0     ],   
    [0.0,      0.0,      0.17453 ],   
    [0.17453,  0.0,      0.0     ],   
    [0.0,      0.17453,  0.0     ],   
]

# t_g2b: translation of the gripper w.r.t. base (meters)
t_g2b = [
    [0.10,  0.00,  0.20],  
    [0.12,  0.02,  0.22],  
    [0.08, -0.03,  0.25],  
    [0.15,  0.01,  0.18],  
]

# rvec_t2c: rotation of the board w.r.t. camera (Rodrigues vec, radians)
rvec_t2c = [
    [0.0,     0.0,     0.26180],  
    [0.0,     0.0,     0.52360],  
    [0.17453, 0.0,     0.26180],  
    [0.0,     0.17453, 0.34907],  
]

# t_t2c: translation of the board w.r.t. camera (meters)
t_t2c = [
    [-0.10,  0.02,  0.50],  
    [-0.12,  0.01,  0.50],
    [-0.09, -0.02,  0.52],
    [-0.11,  0.03,  0.48],
]


In [23]:
import numpy as np, cv2 as cv

# Convert rvec lists into rotation matrices, and make (3,1) translation vectors
R_g2b = [cv.Rodrigues(np.array(rv, np.float64))[0] for rv in rvec_g2b]
t_g2b = [np.array(tv, np.float64).reshape(3,1)     for tv in t_g2b]

R_t2c = [cv.Rodrigues(np.array(rv, np.float64))[0] for rv in rvec_t2c]
t_t2c = [np.array(tv, np.float64).reshape(3,1)     for tv in t_t2c]

R_cam2gripper, t_cam2gripper = cv.calibrateHandEye(
    R_gripper2base=R_g2b, t_gripper2base=t_g2b,
    R_target2cam=R_t2c,   t_target2cam=t_t2c,
    method=cv.CALIB_HAND_EYE_DANIILIDIS
)

# Use pose 0 as "current" gripper->base:
T_BG_now = np.eye(4)
T_BG_now[:3,:3] = R_g2b[0]
T_BG_now[:3, 3] = np.array(t_g2b[0]).reshape(3)

print("Rotation matrix, R_cam2gripper:\n", R_cam2gripper)
print("Traslation vector, t_cam2gripper:\n", t_cam2gripper)
rotation = Rotation.from_matrix(R_cam2gripper)
quaternion = rotation.as_quat()
print("Quaternion based on rotation matrix from cam2gripper: \n",quaternion)


Rotation matrix, R_cam2gripper:
 [[-0.88020223 -0.16934094  0.44335953]
 [ 0.05307978 -0.96344187 -0.26260676]
 [ 0.47162121 -0.20761363  0.85701226]]
Traslation vector, t_cam2gripper:
 [[ 0.52819628]
 [-2.87262804]
 [-0.61381073]]
Quaternion based on rotation matrix from cam2gripper: 
 [ 0.23781683 -0.12221713  0.96185451  0.05781038]


In [ ]:
# Use pose 0 as "current" gripper->base:
T_BG_now = np.eye(4)
T_BG_now[:3,:3] = R_g2b[0]
T_BG_now[:3, 3] = np.array(t_g2b[0]).reshape(3)
print(T_BG_now)

# Build camera->gripper, then camera->base:
T_GC = np.eye(4); T_GC[:3,:3]=R_cam2gripper; T_GC[:3,3]=t_cam2gripper.reshape(3)
T_BC = T_BG_now @ T_GC

# Convert a camera point (meters) to base:
p_cam = np.array([0.05, 0.00, 0.40, 1.0])
p_base = (T_BC @ p_cam)[:3]



print("p_base:", p_base)




[[1.  0.  0.  0.1]
 [0.  1.  0.  0. ]
 [0.  0.  1.  0.2]
 [0.  0.  0.  1. ]]
p_base: [ 0.76152998 -2.97501676 -0.04742477]
